In [3]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adadelta
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from collections import defaultdict
import timeit

In [37]:
df_train = pd.read_csv('data/X_train_processed.csv')
df_test = pd.read_csv('data/X_test_processed.csv')
y_train = pd.read_csv('data/y_train_processed.csv', names=['target'])
df_train = pd.concat([df_train, y_train], axis=1) 

In [38]:
df_train.head()

,cosine similarity (CS),total length (TL),difference length (DL),sim pos tags (POS),POS*DL,POS*TL,POS*POS*TL,POS*POS*DL,common words (CW),CW*POS,...,wc_ratio_unique_stop,same_start_word,same_last_word,char_diff,char_ratio,char_diff_unique_stop,q1_q2_intersect,qid1_max_kcore,qid2_max_kcore,target
0,0.516971,35.0,15.0,0.333333,5.000000,11.666667,3.888889,1.666667,1.0,0.333333,...,2.00,0.0,0.0,13.0,2.444444,13.0,0.0,0.0,0.4,0
1,0.499859,58.0,14.0,0.200000,2.800000,11.600000,2.320000,0.560000,2.0,0.400000,...,1.50,0.0,0.0,12.0,1.631579,12.0,0.0,0.0,0.0,0
2,0.690886,97.0,33.0,0.333333,11.000000,32.333333,10.777778,3.666667,3.0,1.000000,...,0.50,0.0,1.0,28.0,0.500000,24.0,0.0,1.0,1.0,1
3,0.931071,35.0,7.0,0.142857,1.000000,5.000000,0.714286,0.142857,3.0,0.428571,...,0.75,1.0,1.0,6.0,0.666667,6.0,3.0,1.0,1.0,1
4,0.238352,69.0,5.0,0.272727,1.363636,18.818182,5.132231,0.371901,2.0,0.545455,...,1.00,0.0,0.0,4.0,0.875000,0.0,0.0,0.0,0.0,0


In [39]:
df_test.head()

,cosine similarity (CS),total length (TL),difference length (DL),sim pos tags (POS),POS*DL,POS*TL,POS*POS*TL,POS*POS*DL,common words (CW),CW*POS,...,wc_diff_unique_stop,wc_ratio_unique_stop,same_start_word,same_last_word,char_diff,char_ratio,char_diff_unique_stop,q1_q2_intersect,qid1_max_kcore,qid2_max_kcore
0,0.635388,48.0,4.0,0.111111,0.444444,5.333333,0.592593,0.049383,3.0,0.333333,...,1.0,0.80,1.0,0.0,3.0,0.863636,3.0,5.0,1.0,1.0
1,1.000000,24.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,0.000000,...,0.0,1.00,1.0,1.0,0.0,1.000000,0.0,18.0,1.0,1.0
2,1.000000,44.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,4.0,0.000000,...,0.0,1.00,1.0,1.0,0.0,1.000000,0.0,18.0,1.0,1.0
3,0.422159,77.0,5.0,0.090909,0.454545,7.000000,0.636364,0.041322,3.0,0.272727,...,1.0,1.20,0.0,1.0,6.0,0.837838,6.0,7.0,1.0,1.0
4,0.615007,47.0,9.0,0.555556,5.000000,26.111111,14.506173,2.777778,2.0,1.111111,...,1.0,1.25,0.0,1.0,8.0,1.500000,8.0,11.0,1.0,1.0


In [40]:
Train, Validation = train_test_split(df_train, test_size=0.2)

In [41]:
Y_train = Train['target']
Y_validation = Validation['target']

In [23]:
featuresToTake = [
    'cosine similarity (CS)',
    'lda_1',
    'lda_2',
    'freq_q1',
    'freq_q2',
    'jaro_distance',
    'total length (TL)',
    'common words (CW)',
    'q1_q2_intersect',
    'q1_q2_intersect',
    'difference length (DL)',
]

In [42]:
nbSample, nbFeatures = Train[featuresToTake].shape
nbSample, nbFeatures

(64080, 11)

In [43]:
top_words = nbSample
max_review_length = nbFeatures
embedding_vecor_length = 32

In [44]:
featuresToTake

['cosine similarity (CS)',
 'lda_1',
 'lda_2',
 'freq_q1',
 'freq_q2',
 'jaro_distance',
 'total length (TL)',
 'common words (CW)',
 'q1_q2_intersect',
 'q1_q2_intersect',
 'difference length (DL)']

In [45]:
X_train = Train[featuresToTake].values
X_Validation = Validation[featuresToTake].values
X_test = df_test[featuresToTake].values

In [46]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=128, kernel_size=8, padding='same', activation='tanh'))
model.add(Dropout(0.4))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
model.fit(X_train, Y_train.values, epochs=10, batch_size=128, verbose=1)

Epoch 1/10
64080/64080 [==============================] - 20s 319us/step - loss: 0.2678 - acc: 0.8851

In [48]:
predictions = model.predict_proba(X_test)

In [28]:
predictions

array([[0.9914658 ],
       [0.9993697 ],
       [0.9994518 ],
       ...,
       [0.9910761 ],
       [0.11442295],
       [0.355992  ]], dtype=float32)

In [49]:
with open("submition.csv", 'w') as f:
    f.write("Id,Score\n")
    for i in range(len(predictions)):
        f.write(str(i)
                +','
                +str(predictions[i][0])+'\n')